In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python import debug as tf_debug

In [3]:
PATH="data/airbus_ship/"

In [4]:
masks_df = pd.read_csv(f"{PATH}train_ship_segmentations_v2.csv")
masks_df.describe()

,ImageId,EncodedPixels
count,231723,81723
unique,192556,81722
top,e2615fb70.jpg,43801 1 44567 4 45334 5 46100 8 46867 9 47636 ...
freq,15,2


In [5]:
masks_df_nona = masks_df.dropna()
masks_df_nona.describe()

,ImageId,EncodedPixels
count,81723,81723
unique,42556,81722
top,e2615fb70.jpg,43801 1 44567 4 45334 5 46100 8 46867 9 47636 ...
freq,15,2


Create new column: ShipPres

In [6]:
masks_df["ShipPres"] = masks_df["EncodedPixels"].apply(lambda x: 0 if pd.isna(x) else 1)
all_yn = masks_df.groupby(["ImageId"])["ShipPres"].agg({"ShipPres": "max"})
ship_pres = all_yn[all_yn["ShipPres"] == 1]
noship_pres = all_yn[all_yn["ShipPres"] == 0]
TOTAL_WSHIPS = len(ship_pres)
ship_pres.head(5)

/home/mtolstokory/anaconda3/envs/42/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,ShipPres
ImageId,
000155de5.jpg,1
000194a2d.jpg,1
00021ddc3.jpg,1
0002756f7.jpg,1
00031f145.jpg,1


Glue together NUM_SAMPLES images with ships and without ships for training:

In [7]:
NUM_SAMPLES = 1000
train_df = pd.concat([ship_pres.sample(NUM_SAMPLES),noship_pres.sample(NUM_SAMPLES)])
train_df = train_df.reindex(np.random.permutation(train_df.index))
train_df = train_df.reset_index()
train_df.head(10)

,ImageId,ShipPres
0,18af9a74d.jpg,0
1,c85965f88.jpg,1
2,3447c0ffc.jpg,1
3,86195f10d.jpg,1
4,1e5f7cddc.jpg,1
5,275c98805.jpg,1
6,26d81d437.jpg,1
7,d36988638.jpg,0
8,d42a50c68.jpg,0
9,8872cdf6c.jpg,1


In [8]:
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [9]:
#important for monitoring metrics in tensorboard
tf.logging.set_verbosity(tf.logging.INFO)

Split dataframe to get validation data:

In [10]:
SPLIT = int((NUM_SAMPLES*2/100.0)*70)
valid_df = train_df.loc[SPLIT:]
train_df = train_df.loc[:SPLIT]
valid_df.head(5)

,ImageId,ShipPres
1400,bd0004835.jpg,1
1401,888b9510f.jpg,0
1402,8979482c3.jpg,0
1403,37db2f40f.jpg,0
1404,2d781d734.jpg,1


In [11]:
def create_train_input_fn(files,labels, batch_size, num_epochs=1,shuffle=True):
    
    def _input_fn():
        # step 1
        #files = files.apply(lambda x: path + x)
        filenames = tf.constant(list(files))
        _labels = tf.constant(list(labels))

        # step 2: create a dataset returning slices of `filenames`
        dataset = tf.data.Dataset.from_tensor_slices((filenames, _labels))

        # step 3: parse every image in the dataset using `map`
        def _parse_function(filename, label):
            image_string = tf.read_file(filename)#tf.strings.join([path,filename])
            image_decoded = tf.image.decode_jpeg(image_string, channels=3,try_recover_truncated=True)
            image = tf.cast(image_decoded, tf.float32)
            return image, label

        dataset = dataset.map(_parse_function)
        dataset = dataset.batch(batch_size).repeat(num_epochs)

        if shuffle:
          dataset = dataset.shuffle(100)
        
        # step 4: create iterator and final input tensor
        image_batch, label_batch = dataset.make_one_shot_iterator().get_next()
        
        #tf.Print(label_batch,[label_batch],message="Image: ")
        
        return image_batch, label_batch
    return _input_fn

In [12]:
def create_predict_input_fn(files, labels, batch_size):
    
    def _input_fn():
        # step 1
        filenames = tf.constant(list(files))
        _labels = tf.constant(list(labels))

        # step 2: create a dataset returning slices of `filenames`
        dataset = tf.data.Dataset.from_tensor_slices((filenames, _labels))

        # step 3: parse every image in the dataset using `map`
        def _parse_function(filename, label):
            image_string = tf.read_file(filename)#tf.strings.join([path,filename])
            image_decoded = tf.image.decode_jpeg(image_string, channels=3,try_recover_truncated=True)
            image = tf.cast(image_decoded, tf.float32)
            return image, label

        dataset = dataset.map(_parse_function)
        dataset = dataset.batch(batch_size)
        
        # step 4: create iterator and final input tensor
        image_batch, label_batch = dataset.make_one_shot_iterator().get_next()
        
        return image_batch, label_batch
    return _input_fn

In [13]:
BATCH_SIZE = 32
NUM_EPOCHS = 5

In [14]:
train_input_fn = create_train_input_fn(train_df["ImageId"].apply(lambda x: f"{PATH}train/" + x),train_df["ShipPres"],
                                       batch_size=BATCH_SIZE)
valid_input_fn = create_predict_input_fn(valid_df["ImageId"].apply(lambda x: f"{PATH}train/" + x),valid_df["ShipPres"],
                                       batch_size=BATCH_SIZE)




In [15]:
#hooks = [tf_debug.LocalCLIDebugHook()]

sess = tf.InteractiveSession()
img_b, label_b = train_input_fn()
print_out = tf.Print(img_b,[img_b],message="Image: ")
print_out.eval()
sess.close()


In [16]:
def conv_model(features, labels, mode):
    XX = tf.reshape(features, [-1,768,768,3])
    Y1 = tf.layers.conv2d(XX, filters=6, kernel_size=[6,6], padding='same', activation=tf.nn.leaky_relu)
    Y2 = tf.layers.conv2d(Y1, filters=12, kernel_size=[5,5], strides=(2,2), padding='same', activation=tf.nn.leaky_relu)
    Y3 = tf.layers.conv2d(Y2, filters=24, kernel_size=[4,4], strides=(2,2), padding='same', activation=tf.nn.leaky_relu)
    Y4 = tf.layers.flatten(Y3)
    Y5 = tf.layers.dense(Y4, units = 200, activation=tf.nn.leaky_relu)
    Y6 = tf.layers.dropout(Y5, rate = 0.1, training=(mode == tf.estimator.ModeKeys.TRAIN))
    logits = tf.layers.dense(Y6,units = 2)

    predictions = {
                    "classes": tf.argmax(input=logits, axis=1),
                    "probabilities": tf.nn.softmax(logits)
    }
    
    #define predict method logic
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)
    
    #sparse_softmax does one-hot automatically
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
    accuracy = tf.metrics.accuracy(labels = labels, predictions = predictions["classes"], name = "acc_op")
    
    
    #define train method logic
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op  = optimizer.minimize(
            loss = loss,
            global_step=tf.train.get_global_step())
        tf.identity(accuracy[1], name='train_accuracy')
        tf.summary.scalar('train_accuracy', accuracy[1])
        eval_metric_ops = {"train_accuracy":accuracy}
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op, eval_metric_ops=eval_metric_ops)
    
    
    
    #define evaluate method logic
    tf.identity(accuracy[1], name='val_accuracy')
    tf.summary.scalar('val_accuracy', accuracy[1])
    eval_metric_ops = {"val_accuracy":accuracy}
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops=eval_metric_ops)

In [17]:
!rm -rf tf_files
OUT_DIR = "./tf_files"

In [18]:
my_cnn_classifier = tf.estimator.Estimator(model_fn=conv_model, 
                                           config=tf.estimator.RunConfig(keep_checkpoint_max=1).replace(save_summary_steps=2),
                                           model_dir=OUT_DIR)
file_writer = tf.summary.FileWriter(OUT_DIR)

def train_and_eval(estimator, num_epochs = NUM_EPOCHS, steps=100):
    for n in range(num_epochs):
        estimator.train(input_fn = train_input_fn,steps = steps)
        estimator.evaluate(input_fn = valid_input_fn)
        
train_and_eval(my_cnn_classifier)

INFO:tensorflow:Using config: {'_model_dir': './tf_files', '_tf_random_seed': None, '_save_summary_steps': 2, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f27abee8358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_ini

Submission file generation(will be implemented in full version):

generator = my_cnn_classifier.predict(input_fn=test_input_fn)
predictions = [next(generator) for i in range(len(test_images))]
classes = [predictions[i]["classes"] for i in range(len(predictions))]

def make_submission_file(classes, filename):
    submission = pd.DataFrame()
    submission["ImageId"] = range(1, 28001)
    submission["Label"] = classes
    submission.set_index("ImageId", inplace=True)
    submission.to_csv(filename)
    
make_submission_file(classes, "shippres.csv")